<a href="https://colab.research.google.com/github/albertjuntak04/Android/blob/master/psgan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-12-16 04:42:35--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   148MB/s    in 0.5s    

2020-12-16 04:42:36 (148 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [ ]:
! conda install -c rdkit rdkit -y

Solving environment: - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - rdkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    bzip2-1.0.8                |       h7b6447c_0          78 KB
    ca-certificates-2020.12.8  |       h06a4308_0         121 KB
    cairo-1.14.12              |       h8948797_3         906 KB
    certifi-2020.12.5          |   py37h06a4308_0         141 KB
    conda-4.9.2                |   py37h06a4308_0         2.9 MB
    fontconfig-2.13.0          |       h9420a91_0         227 KB
    freetype-2.10.4            |       h5ab3b9f_0         596 KB
    glib-2.66.1                |       h92f7085_0         2.9 MB
    icu-58.2                   |       he6710b0_3        10.5 MB
    intel-openmp-2020.2        |              25

In [ ]:
! conda install -c rdkit rdkit -y

Solving environment: \ | / - \ | / - \ done

# All requested packages already installed.



In [ ]:
python --version

NameError: ignored

In [ ]:
!python --version

Python 3.7.6


In [ ]:
# data_io
import os
import numpy as np
from PIL import Image
from PIL.Image import FLIP_LEFT_RIGHT


def image_to_tensor(img):
    '''
    convert image to Theano/Lasagne 3-tensor format;
    changes channel dimension to be in the first position and rescales from [0,255] to [-1,1]
    '''
    tensor = np.array(img).transpose( (2,0,1) )
    tensor = (tensor / 255.)*2 - 1.
    return tensor


def tensor_to_image(tensor):
    '''
    convert 3-tensor to image;
    changes channel to be last and rescales from [-1, 1] to [0, 255]
    '''
    img = np.array(tensor).transpose( (1,2,0) )
    img = (img + 1.)/2 * 255.
    return np.uint8(img)
    

def get_texture_iter(folder, npx=128, batch_size=64, \
                     filter=None, mirror=True):
    '''
    @param folder       iterate of pictures from this folder
    @param npx          size of patches to extract
    @param n_batches    number of batches to yield - if None, it yields forever
    @param mirror       if True the images get augmented by left-right mirroring
    @return a batch of image patches fo size npx x npx, with values in [0,1]
    '''
    HW    = npx
    imTex = []
    files = os.listdir(folder)
    for f in files:
        name = folder + f
        try:
            img = Image.open(name)
            imTex += [image_to_tensor(img)]
            if mirror:
                img = img.transpose(FLIP_LEFT_RIGHT)
                imTex += [image_to_tensor(img)]
        except:
            print ("Image ", name, " failed to load!")

    while True:
        data=np.zeros((batch_size,3,npx,npx))                   # NOTE: assumes 3 channels!
        for i in range(batch_size):
            ir = np.random.randint(len(imTex))
            imgBig = imTex[ir]
            if HW < imgBig.shape[1] and HW < imgBig.shape[2]:   # sample patches
                h = np.random.randint(imgBig.shape[1] - HW)
                w = np.random.randint(imgBig.shape[2] - HW)
                img = imgBig[:, h:h + HW, w:w + HW]
            else:                                               # whole input texture
                img = imgBig
            data[i] = img

        yield data


def save_tensor(tensor, filename):
    '''
    save a 3-tensor (channel, x, y) to image file
    '''
    img = tensor_to_image(tensor)
    img = Image.fromarray(img)
    img.save(filename)

In [ ]:
# tools
# -*- coding: utf-8 -*-
import sys, os
from time import time


def create_dir(folder):
    '''
    creates a folder, if necessary
    '''
    if not os.path.exists(folder):
        os.makedirs(folder)


class TimePrint(object):
    '''
    Simple convenience class to print who long it takes between successive calls to its __init__ function.
    Usage example:
        TimePrint("some text")          -- simply prints "some text"
        <do some stuff here>
        TimePrint("some other text ")   -- prints "some other text (took ?s)", where ? is the time passed since TimePrint("some text") was called
    '''
    t_last = None

    def __init__(self, text):
        TimePrint.p(text)

    @classmethod
    def p(cls, text):
        t = time()
        print (text),
        if cls.t_last!=None:
            print (" (took ", t-cls.t_last, "s)")
        cls.t_last = t
        sys.stdout.flush()

In [ ]:
# config
import os
# from tools import create_dir
# from data_io import get_texture_iter

create_dir('drive/MyDrive/samples')               # create, if necessary, for the output samples 
create_dir('drive/MyDrive/models') 
# home        = os.path.expanduser("~")


def zx_to_npx(zx, depth):
    '''
    calculates the size of the output image given a stack of 'same' padded
    convolutional layers with size depth, and the size of the input field zx
    '''
    # note: in theano we'd have zx*2**depth
    return (zx - 1)*2**depth + 1


class Config(object):
    '''
    wraps all configuration parameters in 'static' variables -- these are not serialized!!
    '''    
    #optimization constants
    lr          = 0.0002                # learning rate of adam
    b1          = 0.5                   # momentum term of adam
    l2_fac      = 1e-8                  # L2 weight regularization factor
    epoch_count = 10                   #how many epochs to do globally    
    k           = 1                     # number of D updates vs G updates
    batch_size  = 8
    epoch_iters =batch_size * 500      #steps inside one epoch 
                 
    ##constructor to define serializable variables, serialized when dumping model
    def __init__(self):    
        ##
        # sampling parameters    
        self.nz_local = 30    
        self.nz_global = 60                 # num of global Z dimensions
        self.nz_periodic = 3                    # num of global Z dimensions
        self.nz_periodic_MLPnodes = 50          # the MLP gate for the neural network
        self.nz          = self.nz_local+self.nz_global+self.nz_periodic*2                   # num of dim for Z at each field position, sum of local, global, periodic dimensions
        self.periodic_affine = False            # if True planar waves sum x,y sinusoids, else axes aligned sinusoids x or y 
        self.zx          = 2                    # number of spatial dimensions in Z (jumlah dimensi spasial di Z)
        self.zx_sample   = 32                   # size of the spatial dimension in Z for producing the samples    
        self.zx_sample_quilt = self.zx_sample/1      # how many tiles in the global dimension quilt for output sampling

        ##
        # network parameters
        self.nc          = 3                     # number of channels in input X (i.e. r,g,b)
        self.gen_ks      = ([(5,5)] * 5)[::-1]   # kernel sizes on each layer - should be odd numbers for zero-padding stuff
        self.dis_ks      = [(5,5)] * 5           # kernel sizes on each layer - should be odd numbers for zero-padding stuff
        self.gen_ls      = len(self.gen_ks)           # num of layers in the generative network
        self.dis_ls      = len(self.dis_ks)           # num of layers in the discriminative network
        self.gen_fn      = [self.nc]+[2**(n+6) for n in range(self.gen_ls-1)]  # generative number of filters
        self.gen_fn      = self.gen_fn[::-1]
        self.dis_fn      = [2**(n+6) for n in range(self.dis_ls-1)]+[1]   # discriminative number of filters        
        self.npx         = zx_to_npx(self.zx, self.gen_ls) # num of pixels width/height of images in X
        
        ##input texture folder
        self.sub_name    = "ulos1"
        self.texture_dir = "drive/MyDrive/data/"
        self.save_name   = self.sub_name+ "_filters%d_npx%d_%dgL_%ddL_%dGlobal_%dPeriodic_%sAffine_%dLocal" % (self.dis_fn[0],self.npx,self.gen_ls, self.dis_ls,self.nz_global,self.nz_periodic,self.periodic_affine ,self.nz_local)
        self.load_name   = None                  # if None, initializing network from scratch
        
        
           
    ## gives back the correct data iterator given class variables -- this way we avoid the python restriction not to pickle iterator objects
    def data_iter(self):
        return get_texture_iter(self.texture_dir, npx=self.npx, mirror=False, batch_size=self.batch_size)

    def print_info(self):
        ##
        # output some information
        print ("Learning and generating samples from zx ", self.zx, ", which yields images of size npx ", zx_to_npx(self.zx, self.gen_ls)) 
        print ("Producing samples from zx_sample ", self.zx_sample, ", which yields images of size npx ", zx_to_npx(self.zx_sample, self.gen_ls)) 
        print ("Saving samples and model data to file ",self.save_name)

In [ ]:
#psgan
import lasagne
import theano
import theano.tensor as T
import numpy as np
from tqdm import tqdm
from time import time
import sys, os
import joblib

# from config import Config
# from tools import TimePrint
# from data_io import get_texture_iter, save_tensor


##
# define shortcuts for lasagne functions
relu        = lasagne.nonlinearities.rectify
lrelu       = lasagne.nonlinearities.LeakyRectify(0.2)
tanh        = lasagne.nonlinearities.tanh
sigmoid     = lasagne.nonlinearities.sigmoid
conv        = lambda incoming, num_filters, filter_size, W, b, nonlinearity: \
                lasagne.layers.Conv2DLayer(incoming, num_filters, filter_size, stride=(2,2), pad='same', W=W, b=b, flip_filters=True, nonlinearity=nonlinearity)
tconv       = lambda incoming, num_filters, filter_size, W, nonlinearity: lasagne.layers.TransposedConv2DLayer(incoming, num_filters, filter_size, stride=(2,2), crop='same', W=W, nonlinearity=nonlinearity)
batchnorm   = lasagne.layers.batch_norm

# bias and weight initializations
w_init      = lasagne.init.Normal(std=0.02)
b_init      = lasagne.init.Constant(val=0.0)
g_init      = lasagne.init.Normal(mean=1.,std=0.02)


def sharedX(X, dtype=theano.config.floatX, name=None):
    return theano.shared(np.asarray(X, dtype=dtype), name=name)

from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=234)

##given parameters from config, calculate the Z noise tensor
## the tensor has channels of type Global, Local, Periodic dimensions
## @param zx spatial size, now implemented only in square shapes
## @param batch_size how many instances in mini-batch
## @param zx_quilt if not None, will set  some parts of the global dims to random values in different spatial regions (tiles), else all global dim. are equal to the same vector spatially
def sample_noise_tensor(config,batch_size,zx,zx_quilt=None):
    Z = np.zeros((batch_size,config.nz,zx,zx))
    Z[:,config.nz_global:config.nz_global+config.nz_local] = np.random.uniform(-1.,1., (batch_size, config.nz_local, zx, zx) )
    
    if zx_quilt is None:
        Z[:,:config.nz_global] = np.random.uniform(-1.,1., (batch_size, config.nz_global, 1, 1) )
    else:
        for i in range(int(int(zx)/int(zx_quilt))):
            for j in range(int(int(zx)/int(zx_quilt))):
                Z[:,:config.nz_global,i*int(zx_quilt):(i+1)*int(zx_quilt), j*int(zx_quilt):(j+1)*int(zx_quilt)]    =np.random.uniform(-1.,1., (batch_size, config.nz_global, 1, 1) )
   
    
    if config.nz_periodic > 0:
        for i,pixel in zip(range(1,config.nz_periodic+1),np.linspace(30,130,config.nz_periodic)):
            band =  np.pi*(0.5*i / float(config.nz_periodic) +0.5   )##initial values for numerical stability            
            ##just horizontal and vertical coordinate indices
            for h in range(zx):
                Z[:, -i*2,:, h] = h * band 
            for w in range(zx):
                Z[:, -i * 2 + 1, w] = w * band 
    return Z
    
class PeriodicLayer(lasagne.layers.Layer):

    def __init__(self,incoming,config,wave_params):
        self.config = config       
        self.wave_params = wave_params
        self.input_layer= incoming
        self.input_shape = incoming.output_shape
        self.get_output_kwargs = []
        self.params = {}
        for p in wave_params:
            self.params[p] = set('trainable')

    ##the frequency gating MLP
    def _wave_calculation(self,Z):
        if self.config.nz_periodic ==0:
            return Z
        nPeriodic = self.config.nz_periodic

        if self.config.nz_global > 0:  # #MLP or directly a weight vector in case of no Global dims
            h = T.tensordot(Z[:, :self.config.nz_global], self.wave_params[0], [1, 0]).dimshuffle(0, 3, 1, 2) + self.wave_params[1].dimshuffle('x', 0, 'x', 'x')
            band0 = (T.tensordot(relu(h),self.wave_params[2], [1, 0]).dimshuffle(0, 3, 1, 2)) + self.wave_params[3].dimshuffle('x', 0, 'x', 'x')  # #moved relu inside
        else:
            band0 = self.wave_params[0].dimshuffle('x', 0, 'x', 'x')
        
        if self.config.periodic_affine:
            band1 = Z[:, -nPeriodic * 2::2] * band0[:, :nPeriodic] + Z[:, -nPeriodic * 2 + 1::2] * band0[:, nPeriodic:2 * nPeriodic]
            band2 = Z[:, -nPeriodic * 2::2] * band0[:, 2 * nPeriodic:3 * nPeriodic] + Z[:, -nPeriodic * 2 + 1::2] * band0[:, 3 * nPeriodic:]
        else:
            band1 = Z[:, -nPeriodic * 2::2] * band0[:, :nPeriodic] 
            band2 = Z[:, -nPeriodic * 2 + 1::2] * band0[:, 3 * nPeriodic:]
        band = T.concatenate([band1 , band2], axis=1)       
        ##random phase added here, use random stream generator
        band += srng.uniform((Z.shape[0],nPeriodic * 2)).dimshuffle(0,1, 'x', 'x') *np.pi*2
        return T.concatenate([Z[:, :-2 * nPeriodic], T.sin(band)], axis=1)

    def get_output_for(self, input, **kwargs):
        return self._wave_calculation(input)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0],input_shape[1]+self.config.nz_periodic*2,input_shape[2],input_shape[3])     


periodic = lambda incoming,config,wave_params: PeriodicLayer(incoming,config,wave_params)

##
# network code
class PSGAN(object):

    def __init__(self, name=None):
        '''
        @static configuration class
        @param name     load stored sgan model
        '''
        
        if name is not None:
            print ("loading parameters from file:",name)

            vals =joblib.load(name)
            self.config = vals["config"]

            print ("global dimensions of loaded config file",self.config.nz_global) 
            
            self.dis_W = [sharedX(p) for p in vals["dis_W"]]
            self.dis_g = [sharedX(p) for p in vals["dis_g"]]
            self.dis_b = [sharedX(p) for p in vals["dis_b"]]
    
            self.gen_W = [sharedX(p) for p in vals["gen_W"]]
            self.gen_g = [sharedX(p) for p in vals["gen_g"]]
            self.gen_b = [sharedX(p) for p in vals["gen_b"]]

            self.wave_params = [sharedX(p) for p in vals["wave_params"]]
            
            ##now overwrite the static config with the correct values
            self.config.gen_ks = []
            self.config.gen_fn = []
            l = len(vals["gen_W"])
            for i in range(l):
                if i==0:
                    self.config.nz = vals["gen_W"][i].shape[0]
                else:
                    self.config.gen_fn +=[vals["gen_W"][i].shape[0]]
                self.config.gen_ks += [(vals["gen_W"][i].shape[2],vals["gen_W"][i].shape[3])]
            self.config.nc = vals["gen_W"][i].shape[1]
            self.config.gen_fn +=[self.config.nc]

            self.config.dis_ks = []
            self.config.dis_fn = []
            l = len(vals["dis_W"])
            for i in range(l):
                self.config.dis_fn +=[vals["dis_W"][i].shape[1]]   
                self.config.dis_ks += [(vals["gen_W"][i].shape[2],vals["gen_W"][i].shape[3])]             

            self._setup_gen_params(self.config.gen_ks, self.config.gen_fn)
            self._setup_dis_params(self.config.dis_ks, self.config.dis_fn)
        else:
            self.config = Config()

            self._setup_gen_params(self.config.gen_ks, self.config.gen_fn)
            self._setup_dis_params(self.config.dis_ks, self.config.dis_fn)
            ##
            # sample the initial weights and biases
            self._sample_initials()
            
            ##setup the initial MLP frequency gating weights
            self._setup_wave_params()

        self._build_sgan()


    def save(self,name):
        print ("saving PSGAN parameters in file: ", name)
        vals = {}
        vals["config"] = self.config
        vals["dis_W"] = [p.get_value() for p in self.dis_W]
        vals["dis_g"] = [p.get_value() for p in self.dis_g]
        vals["dis_b"] = [p.get_value() for p in self.dis_b]

        vals["gen_W"] = [p.get_value() for p in self.gen_W]
        vals["gen_g"] = [p.get_value() for p in self.gen_g]
        vals["gen_b"] = [p.get_value() for p in self.gen_b]

        vals["wave_params"] = [p.get_value() for p in self.wave_params]
        
        joblib.dump(vals,name,True)

    
    def _setup_wave_params(self):
        '''
        set up the parameters of the periodic dimensions, i.e. the weigts of the gating MLP
        '''

        if self.config.nz_periodic:
            nPeriodic = self.config.nz_periodic
            nperiodK = self.config.nz_periodic_MLPnodes
            if self.config.nz_global >0 and nperiodK>0:##K is hidden nodes layer; MLP depending on global dimensions
                lin1 =  sharedX( g_init.sample( (self.config.nz_global,nperiodK)))
                bias1 = sharedX( g_init.sample( (nperiodK)))
                lin2 =  sharedX( g_init.sample( (nperiodK,nPeriodic * 2*2)))
                bias2 = sharedX( g_init.sample( (nPeriodic * 2*2)))
                self.wave_params = [lin1,bias1,lin2,bias2]
            else:##in case no global dimensions learn global wave numbers
                bias2 = sharedX( g_init.sample( (nPeriodic * 2*2)))
                self.wave_params = [bias2]
            a = np.zeros(nPeriodic * 2*2)              
            a[:nPeriodic]=1#x
            a[nPeriodic:2*nPeriodic]=0#y
            a[2*nPeriodic:3*nPeriodic]=0#x
            a[3*nPeriodic:]=1#y
            self.wave_params[-1].set_value(np.float32(a)) 
        else:
            self.wave_params = []

    def _setup_gen_params(self, gen_ks, gen_fn):
        '''
        set up the parameters, i.e. filter sizes per layer and depth, of the generator
        '''
        ## 
        # setup generator parameters and sanity checks
        if gen_ks==None:
            self.gen_ks = [(5,5)] * 5   # set to standard 5-layer net
        else:
            self.gen_ks = gen_ks

       
        self.gen_depth = len(self.gen_ks)

        if gen_fn!=None:
            assert len(gen_fn)==len(self.gen_ks), 'Layer number of filter numbers and sizes does not match.'
            self.gen_fn = gen_fn
        else:
            self.gen_fn = [64] * self.gen_depth
    

    def _setup_dis_params(self, dis_ks, dis_fn):
        '''
        set up the parameters, i.e. filter sizes per layer and depth, of the discriminator
        '''
        ##
        # setup discriminator parameters
        if dis_ks==None:
            self.dis_ks = [(5,5)] * 5   # set to standard 5-layer net
        else:
            self.dis_ks = dis_ks

        self.dis_depth = len(dis_ks)

        if dis_fn!=None:
            assert len(dis_fn)==len(self.dis_ks), 'Layer number of filter numbers and sizes does not match.'
            self.dis_fn = dis_fn
        else:
            self.dis_fn = [64] * self.dis_depth

    def _sample_initials(self):
        '''
        sample the initial weights and biases and push them back to internal lists
        '''
        self.dis_W = []
        self.dis_b = []
        self.dis_g = []


        self.dis_W.append( sharedX( w_init.sample( (self.dis_fn[0], self.config.nc, self.dis_ks[0][0], self.dis_ks[0][1]) )) )
        for l in range(self.dis_depth-1):
            self.dis_W.append( sharedX( w_init.sample( (self.dis_fn[l+1], self.dis_fn[l], self.dis_ks[l+1][0], self.dis_ks[l+1][1]) ) ) )
            self.dis_b.append( sharedX( b_init.sample( (self.dis_fn[l+1]) ) ) )
            self.dis_g.append( sharedX( g_init.sample( (self.dis_fn[l+1]) ) ) )
    
        self.gen_b = []
        self.gen_g = []
        for l in range(self.gen_depth-1):
            self.gen_b += [sharedX( b_init.sample( (self.gen_fn[l]) ) ) ]
            self.gen_g += [sharedX( g_init.sample( (self.gen_fn[l]) ) ) ]

        self.gen_W = []
        
        last = self.config.nz
        for l in range(self.gen_depth-1):
            self.gen_W +=[sharedX( w_init.sample((last,self.gen_fn[l], self.gen_ks[l][0],self.gen_ks[l][1])))]
            last=self.gen_fn[l]

        self.gen_W +=[sharedX( w_init.sample((last,self.gen_fn[-1], self.gen_ks[-1][0],self.gen_ks[-1][1])))]   

    def _spatial_generator(self, inlayer):
        '''
        creates a PSGAN generator network
        @param  inlayer     Lasagne layer
        '''
        layers  = [inlayer]
        layers.append(periodic(inlayer,self.config,self.wave_params))
        for l in range(self.gen_depth-1):
            layers.append( batchnorm(tconv(layers[-1], self.gen_fn[l], self.gen_ks[l],self.gen_W[l], nonlinearity=relu),gamma=self.gen_g[l],beta=self.gen_b[l]) )
        output  = tconv(layers[-1], self.gen_fn[-1], self.gen_ks[-1],self.gen_W[-1] , nonlinearity=tanh)

        return output

    def _spatial_discriminator(self, inlayer):
        '''
        creates a PSGAN discriminator network
        @param  inlayer     Lasagne layer
        '''
        layers  = [inlayer]
        layers.append( conv(layers[-1], self.dis_fn[0], self.dis_ks[0], self.dis_W[0], None, nonlinearity=lrelu) )
        for l in range(1,self.dis_depth-1):
            layers.append( batchnorm(conv(layers[-1], self.dis_fn[l], self.dis_ks[l], self.dis_W[l],None,nonlinearity=lrelu),gamma=self.dis_g[l-1],beta=self.dis_b[l-1]) )
        output = conv(layers[-1], self.dis_fn[-1], self.dis_ks[-1], self.dis_W[-1], None, nonlinearity=sigmoid)

        return output


    def _build_sgan(self):
        ##
        # network
        Z               = lasagne.layers.InputLayer((None,self.config.nz,None,None))   # leave batch_size and shape unspecified for now
        X               = lasagne.layers.InputLayer((self.config.batch_size,self.config.nc,self.config.npx,self.config.npx))   # leave batch_size and shape unspecified for now

        gen_X           = self._spatial_generator(Z)
        d_real          = self._spatial_discriminator(X)
        d_fake          = self._spatial_discriminator(gen_X)

        prediction_gen  = lasagne.layers.get_output(gen_X)
        prediction_real = lasagne.layers.get_output(d_real)
        prediction_fake = lasagne.layers.get_output(d_fake)

        params_g        = lasagne.layers.get_all_params(gen_X, trainable=True)
        params_d        = lasagne.layers.get_all_params(d_real, trainable=True)

        ##
        # objectives
        l2_gen          = lasagne.regularization.regularize_network_params(gen_X, lasagne.regularization.l2)
        l2_dis          = lasagne.regularization.regularize_network_params(d_real, lasagne.regularization.l2)

        obj_d= -T.mean(T.log(1-prediction_fake)) - T.mean( T.log(prediction_real)) + self.config.l2_fac * l2_dis
        obj_g= -T.mean(T.log(prediction_fake)) + self.config.l2_fac * l2_gen

        ##
        # updates
        updates_d       = lasagne.updates.adam(obj_d, params_d, self.config.lr, self.config.b1)
        updates_g       = lasagne.updates.adam(obj_g, params_g, self.config.lr, self.config.b1)
        
       

        # ##
        # # theano functions
        TimePrint("Compiling the network...\n")
        self.train_d    = theano.function([X.input_var, Z.input_var], obj_d, updates=updates_d, allow_input_downcast=True)
        TimePrint("Discriminator done.")
        self.train_g    = theano.function([Z.input_var], obj_g, updates=updates_g, allow_input_downcast=True)
        TimePrint("Generator done.")
        self.generate   = theano.function([Z.input_var], prediction_gen, allow_input_downcast=True)
        TimePrint("generate function done.")

In [ ]:
pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip


     | 231 kB 234 kB/s
  Created wheel for Lasagne: filename=Lasagne-0.2.dev1-py3-none-any.whl size=122798 sha256=76765f7fcdf3334e91a073d4655492c2e463bd4b77bab780e27402fefbb35bd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rho3n_0s/wheels/b6/a5/97/c657632d2b7fcff539623ea56996e09ec3c83c871e25a62cc5
Successfully built Lasagne


In [ ]:
if __name__=="__main__":   
    psgan        = PSGAN()
    c = psgan.config           
    c.print_info()
    ##
    # sample used just for visualisation
    z_sample        = sample_noise_tensor(c,1,c.zx_sample,c.zx_sample_quilt)
    epoch           = 0
    tot_iter        = 0
    gd_cost = []

    while epoch < c.epoch_count:
        epoch       += 1
        print("Epoch %d" % epoch)

        Gcost = []
        Dcost = []

        iters = c.epoch_iters / c.batch_size
        for it, samples in enumerate(tqdm(c.data_iter(), total=iters)):
            if it >= iters:
                break
            tot_iter+=1

            # random samples for training
            Znp = sample_noise_tensor(c,c.batch_size,c.zx) 

            if tot_iter % (c.k+1) == 0:
                cost = psgan.train_g(Znp)
                Gcost.append(cost)
            else:
                cost = psgan.train_d(samples,Znp)
                Dcost.append(cost)

        print("Gcost=", np.mean(Gcost), "  Dcost=", np.mean(Dcost))
        gd_cost.append([np.mean(Gcost), np.mean(Dcost)])
        
        slist = []
        for img in samples:
            slist +=[img]
        img = np.concatenate(slist,axis=2)        
        # save_tensor(img, 'samples/minibatchTrue_%s_epoch%d.jpg' % (c.save_name,epoch))

        samples =  psgan.generate(Znp)
        slist = []
        for img in samples:
            slist +=[img]
        img = np.concatenate(slist,axis=2)
        # save_tensor(img, 'samples/minibatchGen_%s_epoch%d.jpg' % (c.save_name,epoch))

        data = psgan.generate(z_sample)

        save_tensor(data[0], 'samples/largesample%s_epoch%d_%f_%f.jpg' % (c.save_name,epoch,np.mean(Gcost),np.mean(Dcost)))
        psgan.save('models/%s_epoch%d.psgan'%(c.save_name,epoch))

Compiling the network...



WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Discriminator done.
 (took  76.34541583061218 s)
Generator done.
 (took  23.48599076271057 s)
generate function done.
 (took  7.720543384552002 s)


  0%|          | 0/500.0 [00:00<?, ?it/s]

Learning and generating samples from zx  2 , which yields images of size npx  33
Producing samples from zx_sample  32 , which yields images of size npx  993
Saving samples and model data to file  ulos1_filters64_npx33_5gL_5dL_60Global_3Periodic_FalseAffine_30Local
Epoch 1


FileNotFoundError: ignored

In [15]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: ignored